# Concert Tweet Classifier

## Import Necesary Packages

In [7]:
import sparknlp
from pyspark.sql.types import *

## Start the spark-NLP session

In [3]:
spark = sparknlp.start()

In [4]:
spark.sparkContext.defaultParallelism

4

## Read the data

In [14]:
# set the schemas
tweet_schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("t_id", IntegerType(), True),
    StructField("t_text", StringType(), True),
    StructField("t_dt", StringType(), True)
    ])

user_schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("user_loc", StringType(), True)
])

In [15]:
# read the data
tweets_test = spark.read.format("csv") \
                    .option("delimiter", "\t") \
                    .option("schema", tweet_schema) \
                    .load('../../data/test_set_tweets.txt')

In [16]:
tweets_training = spark.read.format("csv") \
                    .option("delimiter", "\t") \
                    .option("schema", tweet_schema) \
                    .load('../../data/training_set_tweets.txt')

In [17]:
users_test = spark.read.format("csv") \
                    .option("delimiter", "\t") \
                    .option("schema", user_schema) \
                    .load('../../data/test_set_users.txt')

In [18]:
users_training = spark.read.format("csv") \
                    .option("delimiter", "\t") \
                    .option("schema", user_schema) \
                    .load('../../data/training_set_users.txt')

Since we don't have a target variable, these test/train splits are not particularly useful, but a vestige of the original data set and purpose.

## Rough Classifier: contains the word concert